In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr

# Parameters
n_people = 116
n_f_human = 33
n_f_AI = 33
n_steps = 1000
step_size = 0.1
dim = 50  # dimensionality of the sphere
beta_range = np.linspace(0, 2.0, 20)

# Sphere projection
def project_to_sphere(v):
    return v / np.linalg.norm(v)

# Initialize each group on the unit sphere
def init_population(n_vectors, dim):
    return np.array([project_to_sphere(np.random.normal(0, 1, dim)) for _ in range(n_vectors)])

# Hamiltonians
def H_human(X, F_human, J_human):
    return (1 / np.sqrt(n_people + n_f_human)) * np.sum(J_human * (X @ F_human.T))

def H_AI(X, F_AI, J_AI):
    return (1 / np.sqrt(n_people + n_f_AI)) * np.sum(J_AI * (X @ F_AI.T))

# Metropolis update
def metropolis_update(X, F_human, F_AI, J_human, J_AI, beta_human, beta_AI):
    X_new = np.array([project_to_sphere(x + step_size * np.random.normal(0, 1, x.shape[0])) for x in X])
    F_human_new = np.array([project_to_sphere(f + step_size * np.random.normal(0, 1, f.shape[0])) for f in F_human])
    F_AI_new = np.array([project_to_sphere(f + step_size * np.random.normal(0, 1, f.shape[0])) for f in F_AI])

    H_old = beta_human * H_human(X, F_human, J_human) + beta_AI * H_AI(X, F_AI, J_AI)
    H_new = beta_human * H_human(X_new, F_human_new, J_human) + beta_AI * H_AI(X_new, F_AI_new, J_AI)

    if np.random.rand() < np.exp(H_new - H_old):
        return X_new, F_human_new, F_AI_new
    else:
        return X, F_human, F_AI

# Overlap between two sets of vectors
def compute_overlap(A, B):
    return np.mean([np.dot(a, b) for a, b in zip(A, B)])

# Initialize result matrices
energy_matrix = np.zeros((len(beta_range), len(beta_range)))
overlap_people_AI = np.zeros((len(beta_range), len(beta_range)))
overlap_people_human = np.zeros((len(beta_range), len(beta_range)))
overlap_diff = np.zeros((len(beta_range), len(beta_range)))

# Main simulation loop
for i, beta_human in enumerate(tqdm(beta_range, desc="β_human")):
    for j, beta_AI in enumerate(beta_range):
        # Initialize system
        X = init_population(n_people, dim)
        F_human = init_population(n_f_human, dim)
        F_AI = init_population(n_f_AI, dim)
        J_human = np.random.normal(0, 1, (n_people, n_f_human))
        J_AI = np.random.normal(0, 1, (n_people, n_f_AI))

        # Burn-in
        for _ in range(200):
            X, F_human, F_AI = metropolis_update(X, F_human, F_AI, J_human, J_AI, beta_human, beta_AI)

        # Sampling
        total_energy = 0
        for _ in range(n_steps):
            X, F_human, F_AI = metropolis_update(X, F_human, F_AI, J_human, J_AI, beta_human, beta_AI)
            H_total = beta_human * H_human(X, F_human, J_human) + beta_AI * H_AI(X, F_AI, J_AI)
            total_energy += H_total

        avg_energy = total_energy / n_steps
        energy_matrix[i, j] = avg_energy

        # Compute overlaps
        q_people_AI = np.mean([np.dot(x, f) for x in X for f in F_AI])
        q_people_human = np.mean([np.dot(x, f) for x in X for f in F_human])
        overlap_people_AI[i, j] = q_people_AI
        overlap_people_human[i, j] = q_people_human
        overlap_diff[i, j] = q_people_AI - q_people_human

# Save the results to CSV
np.savetxt("energy_matrix.csv", energy_matrix, delimiter=",")
np.savetxt("overlap_people_AI.csv", overlap_people_AI, delimiter=",")
np.savetxt("overlap_people_human.csv", overlap_people_human, delimiter=",")
np.savetxt("overlap_diff.csv", overlap_diff, delimiter=",")


β_human:   0%|          | 0/20 [00:00<?, ?it/s]